In [8]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
import logging
from bson import ObjectId
import base64

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from crud_python import CRUD

# Initialize logging
logging.basicConfig(level=logging.INFO)

# Initialize MongoDB connection details
user = 'aacuser'
password = 'your_aacuser_password'
host = 'nv-desktop-services.apporto.com'
port = 31212
db = 'AAC'
collection = 'animals'

# MongoDB connection and data retrieval
try:
    client = MongoClient(f'mongodb://{user}:{password}@{host}:{port}')
    client.admin.command('ismaster')
    crud = CRUD(user, password, host, port, db, collection)
    data = crud.read({})
    df = pd.DataFrame(data)
    df['_id'] = df['_id'].apply(lambda x: str(x) if isinstance(x, ObjectId) else x)
except Exception as e:
    logging.error(f"Error retrieving data: {e}")
    df = pd.DataFrame()

df = df.fillna('Unknown')
df['location_lat'] = pd.to_numeric(df['location_lat'], errors='coerce')
df['location_long'] = pd.to_numeric(df['location_long'], errors='coerce')
df = df.dropna(subset=['location_lat', 'location_long'])

columns = [{"name": i, "id": i} for i in df.columns]

# Initialize the app
app = JupyterDash('Dashboard')
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
app.layout = html.Div([
     html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image',            style={
                'height': '80px',  # Adjust the height to make the image smaller
                'display': 'block', 
                'margin-left': 'auto', 
                'margin-right': 'auto'
            }
),
    html.H3("Dashboard by Christian Cromer"),
    
    dcc.RadioItems(
        id='filter-options',
        options=[
            {'label': 'All', 'value': 'all'},
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
        ],
        value='all',
        inline=True
    ),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=columns,
        page_size=50,
        page_action='custom',
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '100px', 'width': '100px', 'maxWidth': '100px',
            'whiteSpace': 'normal'
        },
        style_header={'backgroundColor': 'rgb(30, 30, 30)', 'color': 'white'}
    ),
    
    html.Div(id='map-id', className='col s12 m6'),
    dcc.Graph(id='second-chart-id')
])

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-options', 'value'), 
     Input('datatable-id', 'page_current'), 
     Input('datatable-id', 'page_size')]
)
def update_table(filter_value, page_current, page_size):
    if page_current is None: page_current = 0
    if page_size is None: page_size = 50
    
    if filter_value == 'all':
        query = {}
    elif filter_value == 'Water Rescue':
        query = {
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    elif filter_value == 'Mountain or Wilderness Rescue':
        query = {
            'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    elif filter_value == 'Disaster or Individual Tracking':
        query = {
            'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        }
    
    filtered_data = crud.read(query)
    filtered_df = pd.DataFrame(filtered_data)
    
    if '_id' in filtered_df.columns:
        filtered_df['_id'] = filtered_df['_id'].apply(lambda x: str(x) if isinstance(x, ObjectId) else x)
    
    start_index = page_current * page_size
    end_index = start_index + page_size
    
    return filtered_df.iloc[start_index:end_index].to_dict('records')

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'), Input('datatable-id', 'selected_rows')]
)
def update_map(viewData, selected_rows):
    if not viewData or selected_rows is None or len(selected_rows) == 0:
        return [html.P("No data available or no row selected.")]
    
    dff = pd.DataFrame.from_dict(viewData)
    
    if dff.empty or selected_rows[0] >= len(dff):
        return [html.P("No valid data available to display on the map.")]
    
    row = selected_rows[0]

    return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']],
                  children=[dl.Tooltip(dff.iloc[row]['breed']), dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row]['name'])])])
    ])]

@app.callback(
    Output('second-chart-id', 'figure'),
    [Input('filter-options', 'value')]
)
def update_second_chart(filter_value):
    if filter_value == 'all':
        query = {}
    elif filter_value == 'Water Rescue':
        query = {
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    elif filter_value == 'Mountain or Wilderness Rescue':
        query = {
            'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    elif filter_value == 'Disaster or Individual Tracking':
        query = {
            'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        }
    
    filtered_data = crud.read(query)
    filtered_df = pd.DataFrame(filtered_data)
    
    fig = px.bar(filtered_df, x='animal_type', color='animal_type', title='Animal Types Distribution')
    return fig

# Run the app
app.run_server(debug=True)


Dash app running on http://127.0.0.1:27736/
